# E07 - 영화 추천

In [173]:
# Movielens 데이터, 영화에 대한 평점 데이터
# 데이터 준비와 전처리

import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [174]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [175]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings = ratings.drop('timestamp', axis = 1)

In [176]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [177]:
# ratings에서 유니크한 영화 갯수
ratings['movie_id'].nunique()

3628

In [178]:
# ratings에서 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [179]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [180]:

my_favorite = [2985, 3912, 3903, 1959, 1864, 3864, 3825]
my_movielist = pd.DataFrame({'user_id': [6041]*7, 'movie_id': my_favorite, 'count':[5]*7})

# ratings = pd.concat([ratings, my_movielist], sort=False)
# ratings.tail(20)

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 '6041'라는 데이터가 없다면
    ratings = ratings.append(my_movielist)
ratings.tail(20)

,user_id,movie_id,count
1000194,6040,1077,5
1000196,6040,549,4
1000197,6040,2020,3
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5


In [181]:
# 고유한 유저, movie_id를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, movie_id indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# print(user_to_idx[6041])    
# print(movie_to_idx[2985])


In [182]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
2,6039,2391,5
3,6039,366,5
4,6039,3165,5
5,6039,1812,5


In [183]:
# CSR matrix 만들기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_movie = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape = (num_user, num_movie))
csr_movie


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836485 stored elements in Compressed Sparse Row format>

In [186]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_movie_transpose = csr_movie.T
csr_movie_transpose

# 모델 훈련
als_model.fit(csr_movie_transpose)

In [187]:
customer, movie = user_to_idx[6041], movie_to_idx[2985]
customer_vector, movie_vector = als_model.user_factors[customer], als_model.item_factors[movie]

np.dot(customer_vector, movie_vector)



0.280871

In [188]:
beautiful = movie_to_idx[3912]
beautiful_vector = als_model.item_factors[beautiful]
np.dot(customer_vector, beautiful_vector)

0.024469342

In [189]:
# als내에 similar_items메서드를 이용하여 비슷한 영화를 찾아보자.

favorite_movie = 1864
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3165, 0.020417266),
 (2703, 0.015922273),
 (1798, 0.014651149),
 (2029, 0.014191367),
 (3587, 0.013937378),
 (3578, 0.013770155),
 (3580, 0.013755293),
 (3577, 0.01375012),
 (3574, 0.013746098),
 (3575, 0.013741321),
 (3573, 0.013738976),
 (3579, 0.013733418),
 (3572, 0.013733158),
 (3583, 0.013732221),
 (3428, 0.013725463)]

In [190]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
a = [idx_to_movie[i[0]] for i in similar_movie]

[movies.loc[movies['movie_id'] == i, 'title'].item() for i in a]

['Sour Grapes (1998)',
 'Love Stinks (1999)',
 'Killer (Bulletproof Heart) (1994)',
 'Critical Care (1997)',
 'Shopping (1994)',
 'Number Seventeen (1932)',
 "Another Man's Poison (1952)",
 'Daens (1992)',
 'Century (1993)',
 "Brother's Kiss, A (1997)",
 'Male and Female (1919)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Neon Bible, The (1995)',
 'Very Natural Thing, A (1974)']

In [191]:
# als내 recommend메서드를 사용하여 내가 좋아할 만한 영화를 추천해보자.
# filter_already_liked_items는 유저가 이미 평가한 아이템은 제외한다.

user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_movie, N=20, filter_already_liked_items=True)
movie_recommended

[(461, 0.2794635),
 (755, 0.20559931),
 (1814, 0.1891422),
 (1099, 0.18350543),
 (865, 0.17385082),
 (1370, 0.1712672),
 (1853, 0.1604266),
 (745, 0.15761332),
 (909, 0.15401235),
 (811, 0.15371561),
 (1318, 0.15368578),
 (1149, 0.15165327),
 (685, 0.1491546),
 (257, 0.1424692),
 (859, 0.14168665),
 (487, 0.1415818),
 (200, 0.13792792),
 (268, 0.13592166),
 (763, 0.13502496),
 (1837, 0.13187481)]

In [192]:
a = [idx_to_movie[i[0]] for i in movie_recommended]

[movies.loc[movies['movie_id'] == i, 'title'].item() for i in a]

['Meet the Parents (2000)',
 'Almost Famous (2000)',
 'Nurse Betty (2000)',
 'Contender, The (2000)',
 'Predator (1987)',
 'Killing Fields, The (1984)',
 'Best in Show (2000)',
 'Talented Mr. Ripley, The (1999)',
 'Space Cowboys (2000)',
 'Thomas Crown Affair, The (1999)',
 'Requiem for a Dream (2000)',
 'Mosquito Coast, The (1986)',
 'Planet of the Apes (1968)',
 'Night of the Living Dead (1968)',
 'Cell, The (2000)',
 'Boat, The (Das Boot) (1981)',
 'Terminator, The (1984)',
 'Poseidon Adventure, The (1972)',
 'Superman II (1980)',
 'Snow Falling on Cedars (1999)']

In [193]:
# 추천 아티스트 중에 Predator을 추천하게 한 나의 데이터의 기여 정도 확인해보자
# als내 explain메서드

predator = movie_to_idx[461]
explain = als_model.explain(user, csr_movie, itemid=predator)

# 추천 콘텐츠 점수 = 기여한 다른 콘텐츠 + 기여도
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(3825, 0.010054740380044544),
 (3903, 0.00760373392809017),
 (2985, 0.005745885632512426),
 (3864, 0.004302674943573462),
 (1864, -0.0001527882782604095),
 (3912, -0.00018418658538744178),
 (1959, -0.005505665439604316)]

내가 좋아하는 영화를 많이 추가할수록 확률이 높아진다.